In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.linalg import block_diag
from scipy.optimize import linprog



In [2]:
def tropical_pol_function(x,pol):
  a=pol[0]
  b=pol[1]
  s= -math.inf
  for i in range(np.shape(a)[0]):
    if s<np.inner(a[i],x)+b[i]:
      s=np.inner(a[i],x)+b[i]
  return(s)


In [3]:
d=2 #dimension

# Defining the polynomials
a, b=np.array([[0,0],[3,3],[6,0]]), np.array([0,0,0])
a_til, b_til=np.array([[1,0],[1,1],[2,1]]), np.array([0,0,0])
#a_hat, b_hat=np.array([[1,2],[-1,1]]),np.array( [2,3])
a_hat, b_hat=np.random.normal(0,1,6).reshape(3,2),np.random.normal(0.0, 1.0,3)*0.03

p_pol=(a,b)
d_pol=(a_til,b_til)
q_pol=(a_hat,b_hat)

# Samples X_sample
N=200
 
X_sample = np.random.multivariate_normal(np.zeros(d),np.identity(d),N)


#X_sample=np.array([[1, 2 ],[3, 4],[3, -4],[-3, 4]])
#print(X_sample)


In [4]:
# Phase 1
def Phase_1_function(X_sample,q_pol):
  a_hat,b_hat=q_pol
  # Initialize sets I_i
  I_i = np.zeros(np.shape(X_sample)[0])
  for i in range(np.shape(X_sample)[0]):
    q_pol_values = np.matmul( a_hat,X_sample[i])+b_hat      
    I_i[i]=np.argmax(q_pol_values )
  return I_i

In [5]:
def function_f_computation(X_sample,p_pol,d_pol):
  f_x_i=np.zeros(np.shape(X_sample)[0])
  for i in range(np.shape(X_sample)[0]):
    f_x_i[i]=tropical_pol_function(X_sample[i],p_pol)-tropical_pol_function(X_sample[i],d_pol)
  return f_x_i

In [6]:
def Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol):
  a_hat, b_hat=q_pol
  s=np.zeros((np.shape(a_hat)[0],np.shape(X_sample)[1]))
  N=np.zeros((np.shape(a_hat)[0]))
  for i in range(np.shape(a_hat)[0]):
    for j in range(np.shape(X_sample)[0]):
      if I_i[j]==i:
        s[i]=s[i]+X_sample[j]
        N[i]=N[i]+1
  return (s, N)

In [7]:
f_x_i=np.zeros(X_sample.shape[0])
for i in range(X_sample.shape[0]):
  f_x_i[i]=tropical_pol_function(X_sample[i],p_pol)-tropical_pol_function(X_sample[i],d_pol)

In [8]:
# Preparing Ain, bin
A1=np.concatenate((X_sample, np.ones((np.shape(X_sample)[0],1))), axis=1)
A2= np.zeros((np.shape(X_sample)[0],np.shape(a)[0]*np.shape(a_til)[0]))
A3=np.zeros((np.shape(a)[0]*np.shape(a_til)[0],np.shape(X_sample)[1]+1))
A4=-np.identity(np.shape(a)[0]*np.shape(a_til)[0])
A12=np.concatenate((A1,A2),axis=1)
A34=np.concatenate((A3,A4),axis=1)
Ain=np.concatenate((A12,A34),axis=0)

bin=np.concatenate((f_x_i,np.zeros((np.shape(a)[0]*np.shape(a_til)[0]) )))
#print(Ain)
#print(bin)

In [9]:
A_right=a.T
Iden_d = np.identity(d)

A_left =-np.concatenate( (Iden_d, np.array([np.zeros(d)]).T) ,axis=1 )
left_mat = A_left 

A_down_right = np.ones(a.shape[0])
beq1=a_til[0]


for i in range(a_til.shape[0]-1):
  A_right=block_diag(A_right,a.T)
  A_left =np.concatenate((A_left,left_mat),axis=0)
  A_down_right = block_diag(A_down_right,np.ones(a.shape[0]))
  
  beq1=np.concatenate((beq1,a_til[i+1]),axis=0)
   

A_down_left=np.zeros((a_til.shape[0], a.shape[1]+1))

A_up=np.concatenate((A_left,A_right),axis=1)
A_down=np.concatenate((A_down_left,A_down_right),axis=1)
Aeq =np.concatenate((A_up,A_down),axis=0)
beq =np.concatenate((beq1,np.ones(a_til.shape[0])),axis=0)


#print('Aeq=',Aeq), print('beq=',beq), print('Ain=',Ain),print('bin=',bin),print('X_sample=',X_sample),print('a_hat=',a_hat),print('a=',a),print('a_til=',a_til)




In [ ]:
#from numpy.lib.index_tricks import c_
I_i=Phase_1_function(X_sample,q_pol)
s,N=Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol)

i=0
c_cost=np.concatenate((s[i],[N[i]],np.zeros((np.shape(a)[0]*np.shape(a_til)[0]) )))
print(c_cost)

optimize=linprog(c=c_cost, A_ub=Ain, b_ub=bin, A_eq=Aeq, b_eq=beq, bounds=None, method='interior-point', callback=None, options=None, x0=None)
ab_vec=optimize
print(ab_vec.success)


In [14]:
for cnt in range(4):
  q_pol = (a_hat,b_hat)
  I_i=Phase_1_function(X_sample,q_pol)
  s,N=Phase_2_linear_Programming_Prep_s_N(X_sample,I_i,f_x_i,q_pol)
  a_hat_new=np.zeros(a_hat.shape)
  b_hat_new=np.zeros(b_hat.shape)
  for i in range(a_til.shape[0]):
    c_cost=-np.concatenate((s[i],[N[i]],np.zeros((np.shape(a)[0]*np.shape(a_til)[0]) )))
    optimize=linprog(c=c_cost, A_ub=Ain, b_ub=bin, A_eq=Aeq, b_eq=beq, bounds=None, method='simplex', callback=None, options=None, x0=None)#interior-point
    ab_vec_struct=optimize
    ab_vec=ab_vec_struct.x
    a_hat_new[i]=ab_vec[0:a_hat.shape[1]]
    b_hat_new[i]=ab_vec[a_hat.shape[1]:a_til.shape[1]+1]
  a_hat=a_hat_new
  b_hat=b_hat_new
  
  
 

In [ ]:
print('a_hat=',a_hat)
print('b_hat=',b_hat)
#print(I_i)